In [75]:
%matplotlib inline
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime as dt, timedelta as td
from tqdm import tqdm
from rwc import controversy_score
from util import build_graph_from_df
from itertools import takewhile, count

In [76]:
dataset="beefban"

In [77]:
g = nx.read_gpickle('data/{}.gpkl'.format(dataset))
df = pd.read_pickle('data/{}.pkl'.format(dataset))
df['created_at'] = list(map(dt.fromtimestamp, df['created_at']))
df.set_index(['created_at'], inplace=True)

In [78]:
times = df.index

In [79]:
timediff = times.max() - times.min()

In [80]:
print('time difference: {}'.format(timediff))

time difference: 1 days 20:23:17


In [81]:
time_step = td(hours=2)
time_window = td(hours=5)

In [ ]:
time_zero = times.min()
multiples = list(takewhile(lambda i: (time_zero + i * time_step + time_window) <= times.max(), count(0)))
print(multiples)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
rwc_list = []
for i in tqdm(multiples):
    start_time = time_zero + i * time_step
    end_time = start_time + time_window
    sub_df = df[(df.index >= start_time) & (df.index < end_time)]
    sub_g = build_graph_from_df(sub_df)
    try:
        rwc_list.append(controversy_score(sub_g))
    except ValueError as e:
        print('For {} - {}'.format(current_time, current_time + time_step))
        print(str(e))
    current_time += time_step


  0%|          | 0/20 [00:00<?, ?it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/cloud-user/code/controversy_detection/venv/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 80%|████████  | 16/20 [00:53<00:10,  2.68s/it]

In [ ]:
pd.Series(rwc_list).plot(style='-o',
                         title='rwc vs time (step size: {} hr, time window: {} hr)'.format(time_step,
                                                                                           time_window))
plt.savefig('figs/{}-rwc-vs-time.png'.format(dataset))

In [ ]:
pd.Series(df.index).hist(bins=50)
plt.savefig('figs/{}-volume.png'.format(dataset))